# 2. CLASSIFICATORE LINEARE SVM CON INPUT DI N-GRAMMI

Classificatore basato su  SVM che prende in input una matrice di features basata su n-grammi di 3 tipi:
1. Caratteri
2. Token
3. Part of Speech

***

### Preprocessing dei dati

Dobbiamo ottenere un vettore di features, basato sulle occorrenze di n-grammi all'interno delle frasi, per ciascun post. 

In [45]:
import pandas as pd
import os

In [48]:
#creiamo un dataframe che abbia 4 colonne, id (identificativo frase), word, lemma
def create_set(type):
    annotated_posts = []
    for doc in os.listdir("../data/UD_annotation"):
        if type in doc: 
            doc_tokens = []
            doc_path = "../data/UD_annotation/" + doc
            for line in open(doc_path, "r", encoding="utf-8"):
                splitted = line.strip().split("\t")
                if splitted[0].isdigit() and "-" not in splitted[0]:
                    word=splitted[1]
                    lemma=splitted[2]
                    pos=splitted[3]
                    new_token = {'word': word, 'lemma': lemma, 'pos': pos}
                    doc_tokens.append(new_token)
            annotated_posts.append(doc_tokens)
    return annotated_posts

annotation_tr = create_set("training")
annotation_ts = create_set("test")


In [50]:
print(annotation_tr[0])

[{'word': 'Le', 'lemma': 'il', 'pos': 'DET'}, {'word': '5', 'lemma': '5', 'pos': 'NUM'}, {'word': 'sgradevoli', 'lemma': 'sgradevole', 'pos': 'ADJ'}, {'word': 'realtà', 'lemma': 'realtà', 'pos': 'NOUN'}, {'word': 'di', 'lemma': 'di', 'pos': 'ADP'}, {'word': 'cui', 'lemma': 'cui', 'pos': 'PRON'}, {'word': 'Berlusconi', 'lemma': 'Berlusconi', 'pos': 'PROPN'}, {'word': 'dovrebbe', 'lemma': 'dovere', 'pos': 'AUX'}, {'word': 'render', 'lemma': 'rendere', 'pos': 'VERB'}, {'word': 'si', 'lemma': 'si', 'pos': 'PRON'}, {'word': 'personalmente', 'lemma': 'personalmente', 'pos': 'ADV'}, {'word': 'conto', 'lemma': 'conto', 'pos': 'NOUN'}, {'word': '<URL>', 'lemma': '<URL>', 'pos': 'PROPN'}, {'word': 'Mario', 'lemma': 'Mario', 'pos': 'PROPN'}, {'word': 'Monti', 'lemma': 'Monti', 'pos': 'PROPN'}, {'word': 'non', 'lemma': 'non', 'pos': 'ADV'}, {'word': 'usa', 'lemma': 'usare', 'pos': 'VERB'}, {'word': 'mezzi', 'lemma': 'mezzo', 'pos': 'ADJ'}, {'word': 'termini', 'lemma': 'termine', 'pos': 'NOUN'}]


In [49]:
#contiamo il numero di post(sample) per training e test
print(f'Training: {len(annotation_tr)}')
print(f'Test: {len(annotation_ts)}')

Training: 3977
Test: 872


In [72]:
def count_ngrams(post, ft_type,n):
    all_ngrams = {}
    allwords = []
    for word in post:
        allwords.append(word[ft_type])
    for i in range(len(allwords)-(n-1)):
        new_ngram = allwords[i:i+n]
        ngram_id = f'{ft_type}_{n}_'+'_'.join(new_ngram)
        if ngram_id not in all_ngrams:
            all_ngrams[ngram_id] = 1
        else:
            all_ngrams[ngram_id]+=1
    return all_ngrams
        

print(count_ngrams(annotation_tr[2], "word", 2))



{'word_2_#la7_ma': 1, 'word_2_ma_perche': 1, "word_2_perche_'": 1, "word_2_'_Mario": 1, 'word_2_Mario_Monti': 1, 'word_2_Monti_non': 1, 'word_2_non_fa': 1, 'word_2_fa_il': 1, 'word_2_il_premier': 1, 'word_2_premier_?': 1, 'word_2_?_Che': 1, 'word_2_Che_persona': 1, 'word_2_persona_competente': 1, 'word_2_competente_e': 1, 'word_2_e_per': 1, 'word_2_per_bene': 1, 'word_2_bene_!': 1}
